## Data pre-preparation code 
this notebook is mainly used for data preparation... 
main goal is to be reproducible for the main notebook!

### Control parameters for the calculations
The below settings how players will be included or excluded based on their games history as well as what it the minumum number of games for a country to be used in the geographical analysis. It also contains the settings for running mode (use url or local files, save / load temporary calculations and whether to use test data instead of the large non-test files) 

In [ ]:
minimum_years_seen = 1  #number of years a player has to have games for analysis
minimum_games = 10 #number of games a player has to have across all the years seen
minimum_games_country = 50 #number of games for a country to be displayed in the geographical analysis

#if set to true only samples will be loaded not the full datasets
testmode = True

#if set to true precalculated data will be loaded from the saved files
use_prepared_data = False

#if true, when not using generated data interim results will be saved
#please note, when doing precalculation data will only be saved locally
save_prepared_data = False

# if true the files will be read / grabbed from the urls - second and third items in the file name lists
# otherwise local sources will be read - source files need to be in the same directory as the notebook
use_urls = True

### Constants, helpers and other functions
This section contains functions that are either reused in multiple places or just don't belong to any of the sections

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

In [ ]:
# folders where source files and the precalculated ones are stored - to be able to manually check contents
url_source_files: 'https://drive.google.com/drive/folders/15wEVKEMiNeuZSfXBMS4B72IcLDnGU-np/'
url_precalced: 'https://drive.google.com/drive/folders/1RMeA5K3y5_QPG10UE3-CDM-lL8iSD3hG/'

#default directory. the environment has to have geopandas installed and configured
#given the kernel may run in a different environment, start:
# jupyter notebook --notebook-dir=/Users/laszlokovari/Documents/personal/leeds/Programming/Assignments/ChessAnalysis/notebook/

#default file names to load and save
#each filename contains 3 values: [local file name, url for full data, url for sample data]
players_fn = ['Players', 'https://drive.google.com/uc?id=1XryDU569ecgU0X573DivDpBJgUzABh3N','https://drive.google.com/uc?id=13qWhq0QfhxVWT3OWct8TK0NOmqwUZqlp']
ratings_early_fn = ['RatingsRanksTitlesearlier',
                    's3://chessanalysis/RatingsRanksTitlesearlier.csv',
                    'https://drive.google.com/uc?id=1L29Q-Gcdl2bEnm-zX_Z4s-wfRrlJmZ3A']

ratings_late_fn = ['Ratingsrankstitleslater',
                   's3://chessanalysis/Ratingsrankstitleslater.csv'
                   ,'s3://chessanalysis/Ratingsrankstitleslater.csv']
eco_codes_fn = ['eco_codes','https://drive.google.com/uc?id=1v8STV2upc5GLp3jBI3mCR0tyHjaaTuPs','https://drive.google.com/uc?id=1JzAbo9vcda8PBMNJlxZi22_AHHEXTpyo']
#games csv file - assumes already converted data from pgn
games_fn = ['games',
            's3://chessanalysis/games.csv',
            'https://drive.google.com/uc?id=18bDYt6VWRrcQ06xJA_bUhGVezy57J7s9'] 
country_codes = ['countries_codes','https://drive.google.com/uc?id=1MLXgBnyamK64TRlM9eiwxou7Lc4KiVSe','https://drive.google.com/uc?id=1HRvSFlK2NbcQ5_pLydx9ivM0FDlD2MNy'] 
    
#file names for precalculated files
ratings_early_grouped_fn = ['ratings_early_grouped', 'https://drive.google.com/uc?id=10eyrCoG7HXEhuoZsipA4hoT8w5ilfnij','https://drive.google.com/uc?id=1YGj6_O6bUWUwef71PlMI6w3cwu_r_KQV']                            
ratings_later_grouped_fn = ['ratings_later_grouped', 'https://drive.google.com/uc?id=19rWxoXo3EZpa2mI21cGI_l4IM1Dfby06','https://drive.google.com/uc?id=1bwlf9L4WkTVCNTOKTcCRVlqyk18odbjk']                            
players_with_rating_fn = ['players_with_rating','https://drive.google.com/uc?id=1an1m6iRDyTZz_q0GKPeojDTXEAIMl-X4','https://drive.google.com/uc?id=146gmSIghbNNrLccjgyHPj5gvHKfnSoY4']
id_year_grouped_games = ['yearly_grouped_games_for_id','https://drive.google.com/uc?id=1pgArvy4RDCbTSIsGqsC-lh4__P2uR9kT','https://drive.google.com/uc?id=1uwdb6VIfeeQJoXHOfMjOInDQXkr9rzid']



In [ ]:
#returns the filename based on whether the code runs in test more or not. By default it will return csv extension
#only relevant when in local mode, urls will contain all the details
def getfilename(fn, extension = 'csv'):
    """
        Returns the filename / url of the input filename list
        if url are used the second or third item of the relevant filename list is returned 
        for non-test / test respectively, 
        otherwise the first item (optionally with _sample added) and the extension is returned for local processing
    """
    if use_urls:
        if testmode:
            return fn[2]
        return fn[1]
    
    if testmode:
        fn = fn + '_sample'
    if extension == 'csv':
        fn = fn + '.csv'
    else:
        fn = fn + '.' + extension
    return fn

In [ ]:
def savefile(filename, df):
    """
        Saves a files as UTF-8 encoded file
        
        Input:  filename, dataframe
        Output: none
    """
    
    df.to_csv(filename, sep=',', encoding = 'utf-8')

In [ ]:
def loadfile(filename, separator = ','):
    """
        Loads a file and returns the dataframe read from it
        It assumes that the filename contains all the necessary details, i.e. url, token, querystring etc.
        
        Input:  filename
        Output: dataframe with the file contents
    """
    
    df = pd.read_csv(filename, sep=separator, encoding = 'utf-8')
    return df
     
    

### Dataframe and chart functions
The below functions are used across multiple dataframes and include methods to add new columns, calculate columns during grouping. This section also contains the chart generation functions as there are many parameters and this keeps the notebook somewhat less cluttered. 

In [ ]:
def add_totals(df):
    df['total_sharp'] = df['sharp_white'] + df['sharp_black']
    df['total'] = df['total_white'] + df['total_black']
    df['sharp_ratio'] = round((df['total_sharp'] / df['total']).astype(float),2)
    df['not_sharp_ratio'] = (1-df['sharp_ratio']).astype(float)


In [ ]:
# grouping function to be used to aggregate game numbers. no year is seen
def grouping_base(x):
    d = {}
    d['sharp_white'] = x['sharp_white'].sum()
    d['total_white'] = x['total_white'].sum()    
    d['sharp_black'] = x['sharp_black'].sum()
    d['total_black'] = x['total_black'].sum()      
    return d
    
def game_grouping(x):
    d = grouping_base(x)      
    return pd.Series(d)

# grouping function with year
def game_grouping_with_year(x):    
    d = grouping_base(x)      
    d['years_seen'] = x['year'].count()
    return pd.Series(d)

In [ ]:
def apply_grouping_rating(x, sequence = 1):
    #grouping function for ratings aggregation
    #grouping by id, taking max rating, first seen, last seen
    columns = { 
        'country' : 'country', 
        'rating' : 'max_rating',
        'games' : 'games',
        'first_seen' : 'first_seen',
        'last_seen' : 'last_seen'}
    
    
    #add a trailing index to keys
    columns = {x: columns[x]+str(sequence) for x in columns }
    
    d = {}
    d[columns['country'] ] = x['Country'].max()
    d[columns['rating']] = x['Rating'].max()
    d[columns['games']] = x['Games'].sum()
    d[columns['first_seen']] = x['RatingDate'].min()
    d[columns['last_seen']] = x['RatingDate'].max()
    return pd.Series(d)

In [ ]:
#grouping for white id & year and black id & year respectively
def game_grouping_white(x):
    #grouping function for yearly game aggregation
    #grouping by id and year, taking count sharp with white 
    
    d = {}
    d['sharp_white'] = x['Sharp_with_white'].sum()
    d['total_white'] = x['Sharp_with_white'].count()    
    return pd.Series(d)

In [ ]:
#grouping for white id & year and black id & year respectively
def game_grouping_black(x):
    #grouping function for yearly game aggregation
    #grouping by id and year, taking count sharp with white     
    d = {}
    d['sharp_black'] = x['Sharp_with_black'].sum()
    d['total_black'] = x['Sharp_with_black'].count()    
    return pd.Series(d)


In [ ]:
#Player and rating consolidation functions

def consolidate_player_country(row):
    if not pd.isna(row['country1']):
        return row['country1']
    else:
        return row['country2']

def consolidate_player_rating(row):
    if not pd.isna(row['max_rating1']):
        if pd.isna(row['max_rating2']) or row['max_rating1'] > row['max_rating2']:
            return row['max_rating1']
        
    return row['max_rating2']

def consolidate_player_games(row):
    if not pd.isna(row['games1']):
        if pd.isna(row['games2']) or row['games1'] > row['games2']:
            return row['games1']
    return row['games2']
    
def consolidate_player_first_seen(row):
    if not pd.isna(row['first_seen1']):
        if pd.isna(row['first_seen2']) or row['first_seen1'] < row['first_seen2']:
            return row['first_seen1']
    return row['first_seen2']

def consolidate_player_last_seen(row):
    if not pd.isna(row['last_seen1']):
        if pd.isna(row['last_seen2']) or row['last_seen1'] > row['last_seen2']:
            return row['last_seen1']
    return row['last_seen2']


In [ ]:
def percent_format_func(value, tick_number):
    # return % display of float
    return str(int(value * 100))+' %'

In [ ]:
#displaying on stack column chart
def display_age_group_charts():
    source = age_variation[['age_at_game','sharp_ratio','not_sharp_ratio','total_sharp','total' ]].sort_values('age_at_game')
    years = source['age_at_game']
    sharp = source['sharp_ratio']
    not_sharp = source['not_sharp_ratio']

    total_sharp = source['total_sharp']
    total = source['total'] # - source['total_sharp']

    bar_width = 0.6
    fig, axs = plt.subplots(2, 1, figsize=(16,16), sharex=True)
    axs[0].bar(years, sharp, bar_width, label = "Sharp games' %", color='deepskyblue', alpha=0.8)
    axs[0].bar(years, not_sharp, bar_width,bottom=sharp, label = "Not sharp games' %", color='plum', alpha=0.6)
    axs[0].legend(frameon=True)
    axs[0].set_xlabel('age (years)')
    axs[0].set_ylabel('ratio of sharp / not sharp games (%)')
    axs[0].yaxis.set_major_formatter(plt.FuncFormatter(percent_format_func))
    axs[0].grid(axis='y', linestyle='dotted', linewidth = 0.4)
    axs[0].use_sticky_edges = False
    axs[0].spines['top'].set_visible(False)
    axs[0].spines['left'].set_visible(False)
    axs[0].spines['right'].set_visible(False)
    axs[0].spines['bottom'].set_visible(False)
    axs[0].tick_params(bottom=False, left=False)
    axs[0].set_title('Ratio of sharp games from age {} to {}'.format(years.min(), years.max()), pad=5, color='#333333',
                 weight='bold')

    axs[1].bar(years, total_sharp, bar_width/2, label = "Sharp games", color='deepskyblue', alpha=0.8)
    axs[1].bar(years, total, bar_width,bottom=0, label = "Total games", color='plum', alpha=0.6)
    axs[1].legend(frameon=True)
    axs[1].set_xlabel('age (years)')
    axs[1].set_ylabel('number of games')
    axs[1].grid(axis='y', linestyle='dotted', linewidth = 0.4)
    axs[1].use_sticky_edges = False
    axs[1].spines['top'].set_visible(False)
    axs[1].spines['left'].set_visible(False)
    axs[1].spines['right'].set_visible(False)
    axs[1].spines['bottom'].set_visible(False)
    axs[1].tick_params(bottom=False, left=False)
    axs[1].set_title('Number of games from age {} to {}'.format(years.min(), years.max()), pad=5, color='#333333',
                 weight='bold')


In [ ]:
def display_top_countries_chart():
    #displaying the 10-15 countries with the sharpest style... and the top 10-15 least sharp

    country_top = country_data[['sharp_ratio','name']].sort_values('sharp_ratio',ascending = False).head(10)
    fig, axs = plt.subplots(2, 1, figsize=(8,12), sharex=True)
    axs[0].barh(country_top['name'], country_top['sharp_ratio'],0.8,  label = "Sharp games' %", color='deepskyblue', alpha=0.8)
    axs[0].invert_yaxis()
    axs[0].xaxis.set_major_formatter(plt.FuncFormatter(percent_format_func))
    axs[0].set_xlabel("Percentage of sharp games", labelpad=10, weight='bold', size=8)
    axs[0].grid(axis='x', linestyle='dotted', linewidth = 0.4)
    axs[0].use_sticky_edges = False
    axs[0].spines['top'].set_visible(False)
    axs[0].spines['left'].set_visible(False)
    axs[0].spines['right'].set_visible(False)
    axs[0].spines['bottom'].set_visible(False)
    axs[0].set_title('Countries playing the sharpest openings', pad=2, color='#333333',weight='bold')
    axs[0].tick_params(axis="x", which="major", bottom="off", top="off", labelbottom="on", left="off", right="on", labelleft="on")


    country_bottom = country_data[['sharp_ratio','name']].sort_values('sharp_ratio').head(10)
    axs[1].barh(country_bottom['name'], country_bottom['sharp_ratio'],0.8,  label = "Sharp games' %", color='plum', alpha=0.8)
    axs[1].invert_yaxis()
    axs[1].xaxis.set_major_formatter(plt.FuncFormatter(percent_format_func))
    axs[1].set_xlabel("Percentage of sharp games", labelpad=10, weight='bold', size=8)
    axs[1].grid(axis='x', linestyle='dotted', linewidth = 0.4)
    axs[1].use_sticky_edges = False
    axs[1].spines['top'].set_visible(False)
    axs[1].spines['left'].set_visible(False)
    axs[1].spines['right'].set_visible(False)
    axs[1].spines['bottom'].set_visible(False)
    axs[1].set_title('Countries playing the least sharp openings', pad=2, color='#333333',weight='bold')
    axs[1].tick_params(axis="x", which="major", bottom="off", top="off", labelbottom="on", left="off", right="on", labelleft="on")

In [ ]:

def display_countries_classification():
    #dispalying categories on map    
    df_world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    ax = df_world['geometry'].boundary.plot(figsize=(10,6))
    df_world_countries = df_world.merge(country_data, how='left', left_on='name', right_on='Map_country' )
    df_world_countries.plot( column='category', ax=ax, cmap='Greens', 
                         legend=True, missing_kwds={'color': 'lightgrey'})
    ax.set_title('Countries vs their players\' sharp chess opening selection')

### Data loading and generation functions 
generating or loading interim data for dataframe / pandas use

In [ ]:
def load_players():
    """
        Loads the list of chess players from the iput file
        renames id to match with the id of other files
        Cleans some data (i.e. fills empty date of birth) and calculates year born
        
        
        Input:   none
        Oiutput: dataframe of all players
    """
    fn = getfilename(players_fn)
    players_all = loadfile(fn,';')     
            
    #renaming IDNumber to IDnumber to be able to join on it later
    players_all.rename(columns={'IDNumber':'IDnumber'}, inplace = True)
    
    #cleaning some data, i.e. date of birth and calculating age
    #players w/o date of birth -> setting it to 0, so it can be used in non-age related, i.e. geographic analysis
    players_all['DOB'].fillna('0000', inplace= True)
    players_all['year_born'] = players_all['DOB'].str[0:4]
    players_all.sort_values('year_born', ascending = False)
    
    return players_all

In [ ]:
def load_rating_list(data_type):
    
    """
        Loads the rating list for players, cleans (changed decimal to be correct) and formats column types
        Among others these files contains country information, hence will be used in geographical analysis 
        Contains rating which enables further filtering if needed        
        
        Input:  data_type indicating whether early or later data needs to be loaded. 
                accepted values: 'early' 'later'
        Output: dataframe with all ratings in useable format
    """
    if data_type == 'early':
        fn = getfilename(ratings_early_fn)        
    if data_type == 'later':
        fn = getfilename(ratings_late_fn)
        
    #loads the input file, replaces decimal 'point' and returns the dataframe
    dtypes = {'IDnumber': 'str', 'Title': 'str', 'WomensTitle':'str', 'Country':'str', 'Rating':'str', 'Games':'str', 'RatingDate': 'str' }
    df = pd.read_csv(fn, sep=';',dtype=dtypes)
    
    #replacing comma with decimal
    df['Rating'] = df['Rating'].str.replace(',','.')
    df['Games'] = df['Games'].str.replace(',','.')

    #filling rating and games nan with 0s
    df['Rating'].fillna(0.00, inplace=True)
    df['Games'].fillna(0.00, inplace=True)
    
    #converting column types    
    df['IDnumber'] = df['IDnumber'].astype(float)
    df['IDnumber'] = df['IDnumber'].astype(int)
    df['Rating'] = df['Rating'].astype(float)
    df['Games'] = df['Games'].astype(float)    
    
    return df

In [ ]:
def load_eco_codes():    
    """
        Loads the eco codes for opening analysis. 
        The code list contains the assessment if a given variation is considered 'sharp'
        Y/N values are converted to 1/0 to ease calculation
        
        Input:  none
        Output: dataframe with all eco codes and their assessments
    """
    fn = getfilename(eco_codes_fn)
    eco_codes = loadfile(fn) # pd.read_csv('eco_codes.csv')
    #changing sharp values to 1 / 0 for summation
    eco_codes.loc[eco_codes['Sharp_with_white'] == 'Y', 'Sharp_with_white'] = 1
    eco_codes.loc[eco_codes['Sharp_with_white'] == 'N', 'Sharp_with_white'] = 0
    eco_codes.loc[eco_codes['Sharp_with_black'] == 'Y', 'Sharp_with_black'] = 1
    eco_codes.loc[eco_codes['Sharp_with_black'] == 'N', 'Sharp_with_black'] = 0
    return eco_codes

In [ ]:
def load_chess_games():
    """
        Loads the list of chess games, formats columns and fills na values
        Adds a new column 'year' based on the games' date (set to 0000) if unknown
        
        Input:  none
        Output: dataframe of the list of all chess games 
    """
    fn = getfilename(games_fn)
    games_all = loadfile(fn) # pd.read_csv(fn, sep=',', encoding="utf-8")
    
    games_all['ECO'] = games_all['ECO'].str[0:3]
    #filling na values for ids and converting to int so matching can be done
    games_all['WhiteFideId'].fillna(0.0, inplace=True)
    games_all['BlackFideId'].fillna(0.0, inplace=True)
    games_all['WhiteFideId'].replace(' ',0.0, inplace = True)
    games_all['BlackFideId'].replace(' ',0.0, inplace = True)
    games_all['WhiteFideId'] = games_all['WhiteFideId'].astype(float)
    games_all['WhiteFideId'] = games_all['WhiteFideId'].astype(int)
    games_all['BlackFideId'] = games_all['BlackFideId'].astype(float)
    games_all['BlackFideId'] = games_all['BlackFideId'].astype(int)
    
    #adding year and merging with eco codes
    games_all['Date'].fillna(0, inplace=True)
    games_all['year'] = games_all['Date'].str[0:4]
    
    return games_all

In [ ]:
def load_country_list():    
    """
        Loads the list of countries with name and codes. 
        Names are important as geopanda uses names or codes to display data for countries
        however, chess games seem to be having a mix of different country code standards (NOC, IOC, ISO)    
        so the chart uses name 
        
        Input:  none
        Output: dataframe of the list of countries
    """
    fn = getfilename(country_codes)
    country_list = loadfile(fn)

    country_list = country_list[['Country','IOC','Map_country']]
    country_list.rename(columns={'Country':'name', 'IOC':'country'}, inplace = True)

    return country_list


## Players list
loading 3 files:
- Players.csv
- RatingsRanksTitlesearlier.csv
- RatingsRanksTitleslater.csv

building up a single dataframe with:
- player name
- date of birth if known
- year of first seen
- year last seen
- maximum rating over the whole period

Given that the 2 ratings files are huge, an aggregated interim version will be saved.
Those 2 files are gouped by ID

In [ ]:
#fn = getfilename(players_fn)
#players_all = pd.read_csv(fn, ';' ,encoding = 'utf-8')
players_all = load_players()
players_all

In [ ]:
players_all[ (players_all['LastName'] == 'Carlsen') & (players_all['FirstName'] == 'Magnus') ]

## Loading early ratings list...
this list contains 14.2 millions ratings data as person appears or can appear on every single quartely or yearly list as long as he or she is active

Converting this huge list to a more manageable size, where each person (ID) has only one row:
- ID
- Country
- Max rating - so games can be filtered for a given minimum or maximum level if needed
- First seen on the list - to be able to filter people who have enough games 
- Last seen  - to be able to filter people who have enough games 

In [ ]:
#loading ratings list from 1970 to 2016

if use_prepared_data:
    #load data from file
    fn = getfilename(ratings_early_grouped_fn)
    grouped_earlier = loadfile(fn)
else:
    ratings_earlier = load_rating_list('early')
    print(ratings_earlier['Rating'].sum()) #test ratings are read as numbers
    #grouping by ID and aggregating other data points
    grouped_earlier = ratings_earlier.groupby(['IDnumber'], sort=False).apply(apply_grouping_rating)


In [ ]:
grouped_earlier.sort_values('first_seen1') #test grouping result

In [ ]:
#if set, saving the file for later reload, so may not need to regroup again
if save_prepared_data == True:
    fn = getfilename(ratings_early_grouped_fn)
    savefile(fn, grouped_earlier)


## Loading later ratings list...
this list contains 14.2 millions ratings data as person appears or can appear on every single quartely or yearly list as long as he or she is active

Converting this huge list to a more manageable size, where each person (ID) has only one row:

- ID
- Country
- Max rating - so games can be filtered for a given minimum or maximum level if needed
- First seen on the list - to be able to filter people who have enough games
- Last seen - to be able to filter people who have enough games

In [ ]:
if use_prepared_data:
    #load data from file
    fn = getfilename(ratings_later_grouped_fn)
    grouped_later = loadfile(fn)
else:
    ratings_later = load_rating_list('later')
    grouped_later = ratings_later.groupby(['IDnumber'], sort=False).apply(apply_grouping_rating,2)

In [ ]:
grouped_later.sort_values('first_seen2')

In [ ]:
#if set, saving the file for later reload, so may not need to regroup again
if save_prepared_data == True:
    fn = getfilename(ratings_later_grouped_fn)
    savefile(fn, grouped_later)

## Combining player and rating lists
combinig the player database can be essentially used for rating filtering and country mapping -> geographical display and filtering

### Preparing the consolidated player list
Preparing a list with single maximum rating and seen dates taking the min of first seen dates, the max of last seen dates and the max of ratings there is no check on the country data, if the first is set then it's taken, if not, then the second, regardless of value

In [ ]:
grouped_earlier = grouped_earlier.reset_index()
grouped_earlier['IDnumber'] = grouped_earlier['IDnumber'].astype(int)
grouped_later = grouped_later.reset_index()
grouped_later['IDnumber'] = grouped_later['IDnumber'].astype(int)

In [ ]:
#using another checkpoint and stored file if set
if use_prepared_data:
    fn = getfilename(players_with_rating_fn)    
    players_with_rating = loadfile(fn)
else:
    players_with_rating = players_all.merge(grouped_earlier, on='IDnumber', how='left')
    players_with_rating = players_with_rating.merge(grouped_later, on='IDnumber', how='left')
    players_with_rating['country'] = players_with_rating.apply(consolidate_player_country, axis=1)
    players_with_rating['max_rating'] = players_with_rating.apply(consolidate_player_rating, axis=1)
    players_with_rating['games'] = players_with_rating.apply(consolidate_player_games, axis=1)
    players_with_rating['first_seen'] = players_with_rating.apply(consolidate_player_first_seen, axis=1)
    players_with_rating['last_seen'] = players_with_rating.apply(consolidate_player_last_seen, axis=1)
    #dropping old colums
    players_with_rating.drop(columns = ['country1', 'country2', 'max_rating1', 'max_rating2', 'games1', 'games2', 'first_seen1', 'first_seen2', 'last_seen1', 'last_seen2'], inplace = True )
    players_with_rating

In [ ]:
#if set, saving the consolidated player list
if save_prepared_data == True:
    fn = getfilename(players_with_rating_fn)
    savefile(fn,players_with_rating)

## Loading ECO file
ECO classification file is simple table for opening variants denoted by the 3 letter ECO codes. 
For each such code some assessment has been made to see if it is 'sharp' when played by white and by black

In [ ]:
eco_codes = load_eco_codes()
eco_codes

## Loading chess games file
Using caissabase 4.2 million games in pgn format can be loaded, however only a fraction of its content is really needed for this analysis. Therefore the steps to process this file are:
- convert the file to csv by using pgn2data library
- reusing only the 'header' information, game content, moves are discarded
- for every game the following will be used
    - player ids
    - colour - if played with white or black
    - year or date of the game
    - ECO code or classification. This will be used to classify games to be sharp or not sharp

In [ ]:
games_all = load_chess_games()
games_all =  games_all.merge(eco_codes, on='ECO', how='left')
#filtering out variations not in analysis
games_all = games_all[games_all['In_analysis']=='Y']
games_all

In [ ]:
games_white_group = games_all.groupby(['WhiteFideId','year'], sort=False).apply(game_grouping_white)
games_white_group.reset_index(inplace = True)
games_white_group.rename(columns={'WhiteFideId':'IDnumber'}, inplace = True)
games_white_group

In [ ]:
games_black_group = games_all.groupby(['BlackFideId','year'], sort=False).apply(game_grouping_black)
games_black_group.reset_index(inplace = True)
games_black_group.rename(columns={'BlackFideId':'IDnumber'}, inplace = True)
games_black_group

In [ ]:
# the final merged table contains aggregated data for analysis
# it contains grouped game summary for every year and for every player
# given the time it takes to generates, it can be pregen-ed and loaded as well
if use_prepared_data:
    fn = getfilename(id_year_grouped_games)
    games_merged_for_id_year_player = loadfile(fn)
else:
    #joining the 2 dataframes - outer join, there could be players with only one colour
    games_merged_for_id_year = games_white_group.merge(games_black_group, on=['IDnumber','year'], how='outer')
    #filtering players with no id
    games_merged_for_id_year= games_merged_for_id_year[games_merged_for_id_year['IDnumber'] != 0 ]

    #merging player data with aggregated games info. this will then be used to analyze age and country related data
    games_merged_for_id_year_player = games_merged_for_id_year.merge(players_with_rating, on='IDnumber', how='left')
    
    #filtering for unknown players
    games_merged_for_id_year_player = games_merged_for_id_year_player[games_merged_for_id_year_player['LastName'].notnull() ]
    
games_merged_for_id_year_player

In [ ]:
#saving temp file as the aggregation takes quite some time
if save_prepared_data:
    fn = getfilename(id_year_grouped_games)
    savefile(fn, games_merged_for_id_year_player)

## Question 1 - player age variation analysis
this section filters players for date of birth, minimum games and years seen and can be used for age variation display

In [ ]:
#dropping unnecessary columns, filtering on having a year_born value, filling nan
#and calculating age at game for variation analysis
age_variation = games_merged_for_id_year_player[['IDnumber', 'year', 'sharp_white','total_white', 'sharp_black','total_black', 'year_born']]
age_variation = age_variation[ age_variation['year_born'] != '0000' ] 

age_variation['age_at_game'] = age_variation['year'].astype(int) - age_variation['year_born'].astype(int)
age_variation.fillna(0.0, inplace = True)
#age_variation.sort_values('year_born')

In [ ]:
#filtering out players with not enough games or not seen for enough years
#this is done by aggregating games for idnumber and counting the years seen. 
#once this list is there we can join it back to the above table (inner), so only those ids remain 
#that meet the filtering criteria
included_players = age_variation.groupby(['IDnumber'], sort=False).apply(game_grouping_with_year)
included_players['total_games'] = included_players['total_white'] + included_players['total_black']
included_players = included_players[(included_players['years_seen'] >= minimum_years_seen) & (included_players['total_games'] >= minimum_games)]
included_players.reset_index(inplace = True)
#dropping unneeded columns, just need Id, then rejoin the table
included_players = included_players['IDnumber']
age_variation = age_variation.merge(included_players, on='IDnumber', how = 'inner')
age_variation.head()

In [ ]:
#the dataframe still contains a lot of unneeded data for age variation (year, id, details year born)
#need to remove those and group / sum by age
#once grouped need to remove outliers, anything below the age of 6 and above age of 89 is to be removed
age_variation.drop(columns = ['IDnumber', 'year', 'year_born'], inplace = True )
age_variation = age_variation.groupby(['age_at_game'], sort=False).apply(game_grouping)
age_variation.reset_index(inplace = True)

#filtering outliers
age_variation = age_variation[ (age_variation['age_at_game'] >5) & (age_variation['age_at_game'] <90) ]

# add totals and ratios
age_variation.reset_index(inplace=True)
add_totals(age_variation)
age_variation['total_sharp'] = age_variation['sharp_white'] + age_variation['sharp_black']
age_variation['total'] = age_variation['total_white'] + age_variation['total_black']
age_variation['sharp_ratio'] = round((age_variation['total_sharp'] / age_variation['total']).astype(float),2)
age_variation['not_sharp_ratio'] = (1-age_variation['sharp_ratio']).astype(float)
age_variation


In [ ]:
print('Ratio of sharp games for 18-23')
youngs = age_variation[ (age_variation['age_at_game'] >= 18) & (age_variation['age_at_game'] <= 23)] .sort_values('age_at_game')
youngs[['age_at_game','sharp_ratio', 'total']]


In [ ]:
print('Ratio of sharp games for 40-45')
middleage = age_variation[ (age_variation['age_at_game'] >= 40) & (age_variation['age_at_game'] <= 45)] .sort_values('age_at_game')
middleage[['age_at_game','sharp_ratio','total']]

In [ ]:
display_age_group_charts()

## Question 2 - location style differences
this section filters players for country only, not filtering on age, number of games etc, the only requirement is that a game has to have a known player

creating 2 dataframes, one for country - year grouping and one for country only

In [ ]:
# the dataframe for country comparisons will need to have the country column filled, 
# this will unfortunately seriously decrease the available data
# moreover for reasonable data it needs to be filtered for minimum games

country_data = games_merged_for_id_year_player[ games_merged_for_id_year_player['country'].notnull() ][['sharp_white', 'sharp_black', 'total_white', 'total_black','country']]
country_data = country_data.groupby(['country'], sort=False).apply(game_grouping)
add_totals(country_data)
country_data.reset_index(inplace = True)
country_data = country_data[country_data['total'] > minimum_games_country]
country_data

In [ ]:
#loading country code list, removing unneeded columns and merge it based on country code / alpha 3 iso code
country_list = load_country_list()
country_data= country_data.merge(country_list, on='country', how='left')
country_data.sort_values('sharp_ratio', ascending = False)


In [ ]:
print('Top 10 countries with sharpest opening play ')
c = country_data.sort_values('sharp_ratio', ascending = False)
c = c[['country', 'total_sharp','total', 'sharp_ratio','name']]
c.head(10)

In [ ]:
print('Top 10 countries with most positional opening play ')
c.tail(10)

In [ ]:
#each country is classified into one of 5 bands based equal distribution of min / max of country values
max_sharpness = country_data['sharp_ratio'].max()
min_sharpness = country_data['sharp_ratio'].min()
print('ranges is: {} to {}'.format(str(min_sharpness), str(max_sharpness)))

if max_sharpness > min_sharpness:
    #creating five bins dynamically and bin the countries based on these 
    bin_labels = [1,2,3,4,5]
    bin_size = (max_sharpness - min_sharpness) / 5

    bins = []
    for i in range(6):
        bins.append( round(min_sharpness + i*bin_size,2) )

    print('bin size' + str(bin_size))
    print(bins)
    
    #classify - bin countries
    country_data['category'] = pd.cut(country_data['sharp_ratio'], bins = bins, labels = bin_labels )    
else:
    country_data['category'] = 3
    print('cannot classify, not large enough range, setting all countries to 3')

#displaying data - contains bin / category as well now
country_data.head()

In [ ]:
display_top_countries_chart()

In [ ]:
display_countries_classification()